In [ ]:
import pandas as pd
import numpy as np
import random as rand
import matplotlib.pyplot as plt
import numpy.linalg as la
import scipy.linalg as sci
import math
import sklearn.metrics as skm

# setting up the dictionary of letters for the Bonthron approach
letter_dict = {"a":1,"b":2,"c":3, "d":4, "e":5, "f":6, "g":7, "h":8, "i":9, "j":10, "k":11, "l":12, "m":13, "n":14,
               "o": 15, "p":16, "q":17, "r":18, "s":19, "t":20, "u":21, "v":22, "w":23, "x":24, "y":25, "z":26}

def words_to_vectors(words):
  out = []
  for i in words:
    letters = [*i]
    out2 = []
    for j in letters:
      out2.append(letter_dict[j])
    out.append(out2)
  colg_letters = []
  for i in out:
    word = []
    for l in i:
      for j in range(26):
        if l == j+1:
          word.append(1)
        else:
          word.append(0)
    colg_letters.append(word)
    solution= np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
  return solution

def vector_to_word(vector):
  vector1 = (vector).tolist()
  letter1 = vector1[0][0:26]
  letter2 = vector1[0][26:52]
  letter3 = vector1[0][52:78]
  letter4 = vector1[0][78:104]
  letter5 = vector1[0][104:130]
  word = ""
  for i,j in enumerate(letter1):
    if j == 1:
      word = list(letter_dict.keys())[i]
  for i,j in enumerate(letter2):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  for i,j in enumerate(letter3):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  for i,j in enumerate(letter4):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  for i,j in enumerate(letter5):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  return word

def best_word1(solution, all_words):
  """
  solution: remaining solution space
  all_words: the whole action space
  """
  #Take SVD of solutions matrix S, select first left singular vector (dom. eigenvector of SS^T) and find its norm.
  U, S, Vh = np.linalg.svd(solution, full_matrices=True)
  dom1 = np.abs(U[:,0])
  normdom = np.linalg.norm(dom1)
  #initiate maximum angle and empty word out
  word = ""
  angle_min = 90
  #Loop over all words possible playable words find cosine simmilairty and determine if cosines similarity is smaller than
  # the smallest iterated word store angle_min as the new angle and the word as the new word
  for i in range(0,all_words.shape[1]):
    angle1 = np.degrees(np.arccos(np.matmul(np.transpose(all_words[:,i]),dom1)/(np.linalg.norm(all_words[:,i]) * normdom)))
    if np.abs(angle1) < angle_min:
          angle_min = angle1
          word = all_words[:,i]
  #convert vectorized word into normal word
  return vector_to_word(np.transpose(word))

def find_all_char_positions(word: str, char: str):
    positions = []
    pos = word.find(char)
    while pos != -1:
        positions.append(pos)
        pos = word.find(char, pos + 1)
    return positions

guesses = pd.read_csv("valid_guesses.csv") 
solutions = pd.read_csv("valid_solutions.csv")
S = list(solutions["word"]) # list of all potential solutions
G = list(guesses["word"])+S # set of all possible guesses

def coloring(secret,g):
    output = [0] * len(secret)
    counted_pos = set()
    for index, (expected_char, guess_char) in enumerate(zip(secret, g)):
        if expected_char == guess_char:
            output[index] = 2
            counted_pos.add(index)
    for index, guess_char in enumerate(g):
        if guess_char in secret and \
                output[index] != 2:
            positions = find_all_char_positions(word=secret, char=guess_char)
            for pos in positions:
                if pos not in counted_pos:
                    output[index] = 1
                    counted_pos.add(pos)
                    break
    colors = []
    for i in range(len(output)):
        if output[i] == 2:
            colors.append("green")
        elif output[i] == 1:
            colors.append("yellow")
        else:
            colors.append("gray")
    return colors

def elim(S, g, colors):
    """Given a current solution list, a guess, and the coloring that comes up
    when we make that guess, returns the new list of possible solutions after
    applying all of the new information we learn from the coloring of the guess."""
    if g in S:
      S.remove(g)
        
    j = 0
    while j < len(S): # iterating through the remaining words to see which ones get eliminated
      keep = True
      for i in range(len(S[j])):
        keep = keep and not (colors[i] == "green" and S[j][i] != g[i])
        keep = keep and not (colors[i] == "yellow" and g[i] not in S[j])
        keep = keep and not (colors[i] == "yellow" and S[j][i] == g[i])
        keep = keep and not (colors[i] == "yellow" and S[j].count(g[i]) < g[:i+1].count(g[i]))
        keep = keep and not (colors[i] == "gray" and S[j].count(g[i]) >= g.count(g[i]))
      if keep:
          j += 1
      else:
          S.pop(j)
    return S

def eliminate(S,g):
    """Given a current list of potential solutions and a guess, takes a
    random sample of the solutions, and for each solution in the random 
    sample, returns the potential solution list at the start of the next 
    stage if that solution is correct."""
    res = []
    for s in rand.choices(S,k=math.ceil(len(S)/100)): # conditioning on each remaining word being the solution
        res.append(S.copy())
        colors = coloring(s,g)
        elim(res[-1], g, colors)
    return res

In [ ]:
def minimax(S,t):
    """This greedy algorithm, at each stage t, takes a random sample of the
    remaining potential solutions list and chooses as the guess the word from
    that sample that would minimize the maximum size (also based on a random sample, 
    so not exact) of the potential solutions list at the start of the next stage t+1."""
    if len(S) == 1:
        return S[0]
    if t < 6:
        stage_guesses = rand.choices(S,k=math.ceil(len(S)/100))
        lengths = [max([len(sols) for sols in eliminate(S,g)]) for g in stage_guesses] # list of maximal lengths of next solution list for each guess
        return stage_guesses[lengths.index(min(lengths))] # choose the guess whose maximal length of the next solution list is smallest
    else:
        return rand.choice(S)

def min_expected(S,t):
    """This greedy algorithm, at each stage t, takes a random sample of the
    remaining potential solutions list and chooses as the guess the word from
    that sample that would minimize the expected size (also based on a random sample, 
    so not exact) of the potential solutions list at the start of the next stage t+1."""
    if len(S) == 1:
        return S[0]
    if t < 6:
        stage_guesses = rand.choices(S,k=math.ceil(len(S)/100))
        lengths = [sum([len(sols) for sols in eliminate(S,g)]) for g in stage_guesses] # list of sum of lengths of next solution list for each guess
        # taking min sum is equivalent to taking min expectation as  and we assume secret chosen uniformly at random
        return stage_guesses[lengths.index(min(lengths))] # choose the guess whose maximal length of the next solution list is smallest
    else:
        return rand.choice(S)

In [ ]:
# testing the minimax
S = list(solutions["word"])
Sk = S.copy()
solved = []
attempts = []
for secret in Sk:
    S = list(solutions["word"])
    t = 1
    colors = []
    while t < 7 and colors != ["green"] * 5:
        guess = minimax(S,t)
        colors = coloring(secret,guess)
        if colors != ["green"] * 5:
            elim(S,guess,colors)
            t += 1
    solved.append(int(colors == ["green"] * 5))
    if solved[-1]:
        attempts.append(t)

print(sum(solved))
print(sum(solved)/len(solved))
print(sum(attempts)/len(attempts))

In [ ]:
# testing the min_expected
S = list(solutions["word"])
Sk = S.copy()
solved = []
attempts = []
for secret in Sk:
    S = list(solutions["word"])
    t = 1
    colors = []
    while t < 7 and colors != ["green"] * 5:
        guess = min_expected(S,t)
        colors = coloring(secret,guess)
        if colors != ["green"] * 5:
            elim(S,guess,colors)
            t += 1
    solved.append(int(colors == ["green"] * 5))
    if solved[-1]:
        attempts.append(t)

print(sum(solved))
print(sum(solved)/len(solved))
print(sum(attempts)/len(attempts))

In [ ]:
# testing the random algorithm
S = list(solutions["word"])
Sk = S.copy()
solved = []
attempts = []
for secret in Sk:
    S = list(solutions["word"])
    t = 1
    colors = []
    while t < 7 and colors != ["green"] * 5:
        guess = rand.choice(S)
        colors = coloring(secret,guess)
        if colors != ["green"] * 5:
            elim(S,guess,colors)
            t += 1
    solved.append(int(colors == ["green"] * 5))
    if solved[-1]:
        attempts.append(t)

print(sum(solved))
print(sum(solved)/len(solved))
print(sum(attempts)/len(attempts))

In [ ]:
# setting up the Bonthron approach
out = []
for i in G:
  letters = [*i]
  out2 = []
  for j in letters:
    out2.append(letter_dict[j])
  out.append(out2)
colg_letters = [] # this will be the columns of the guess matrix
for i in out:
  word = []
  for l in i:
    for j in range(26):
      if l == j+1:
        word.append(1)
      else:
        word.append(0)
  colg_letters.append(word)

out = []
for i in solutions["word"]:
  letters = [*i]
  out2 = []
  for j in letters:
    out2.append(letter_dict[j])
  out.append(out2)
cols_letters = [] # this will be the columns of the solutions matrix
for i in out:
  word = []
  for l in i:
    for j in range(26):
      if l == j+1:
        word.append(1)
      else:
        word.append(0)
  cols_letters.append(word)

In [ ]:
all_words = colg_letters
all_words = np.transpose(np.asmatrix(np.array([np.array(xi) for xi in all_words])))
guess= np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
poss_sols= np.transpose(np.asmatrix(np.array([np.array(xi) for xi in cols_letters])))

In [ ]:
# We see in the output that the Bonthron algorithm chooses SAINE as the starting guess.
all_words = np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
S = list(pd.read_csv("valid_solutions.csv")["word"])
solutionss = words_to_vectors(S)
guess = best_word1(np.array(solutionss),all_words)
print(guess)

In [ ]:
# testing adapted Bonthron approach
S = list(pd.read_csv("valid_solutions.csv")["word"])
Sk = S.copy()
solved = []
attempts = []
for secret in Sk:
    all_words = np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
    S = list(pd.read_csv("valid_solutions.csv")["word"])
    t = 1
    colors = []
    while t < 7 and colors != ["green"] * 5:
        if t == 1:
            guess = "saine"
        else:
            solutionss = words_to_vectors(S)
            guess = best_word1(np.array(solutionss),all_words)
        colors = coloring(secret,guess)
        if colors != ["green"] * 5:
            elim(S,guess,colors)
            t += 1
    solved.append(int(colors == ["green"] * 5))
    if solved[-1]:
        attempts.append(t)

print(sum(solved))
print(sum(solved)/len(solved))
print(sum(attempts)/len(attempts))

In [ ]:
# best word again but using PCA and cosine similairty
def best_word(solution, all_words):
  # do the SVD
  U, S, Vh = np.linalg.svd(solution, full_matrices=True)
  if len(S)>=3:
    # Calculate the three best rank 1 approximations
    dom1 = np.matmul(U[:,0], Vh[0])*S[0]
    dom2 = np.matmul(U[:,1], Vh[1])*S[1]
    dom3 = np.matmul(U[:,2], Vh[2])*S[2]


    #commented out code that is used for producing plots but not used in best word algortihm
    #dom11 = np.matmul(U[:,3], Vh[3])*S[3]
    #dom21 = np.matmul(U[:,4], Vh[4])*S[4]
    #dom31 = np.matmul(U[:,5], Vh[5])*S[5]
    #dom12 = np.matmul(U[:,6], Vh[6])*S[6]
    #dom22 = np.matmul(U[:,7], Vh[7])*S[7]
    #dom32 = np.matmul(U[:,8], Vh[8])*S[8]


    #Calculating the percent explained
    percent1 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom1))
    percent2 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom2))
    percent3 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom3))
    #commented out code that is used for producing plots but not used in best word algortihm
    #percent11 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom11))
    #percent21 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom21))
    #percent31 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom31))
    #percent12 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom12))
    #percent22 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom22))
    #percent32 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom32))
    # calculating the norm of the rank 1
    normdom1 = np.linalg.norm(dom1[:,0])
    normdom2 = np.linalg.norm(dom2[:,0])
    normdom3 = np.linalg.norm(dom3[:,0])

    #commented out code that is used for producing plots but not used in best word algortihm
    #plt.plot(np.arange(1,10), [percent1, percent2, percent3, percent11, percent21, percent31, percent12, percent22, percent32])
    #plt.plot(np.arange(1,4), [percent1, percent2, percent3])
    #plt.title('Principle components Vs Percent Variance Explained')
    #plt.xlabel('Principle Component')
    #plt.ylabel('Percent Variance Explained')
    #plt.show()

    word = ""
    angle_min = 0
    for i in range(0,all_words.shape[1]):
      angle1 = np.degrees(np.arccos(np.matmul(np.transpose(dom1[:,0]), (all_words[:,i]))/(np.linalg.norm(all_words[:,i]) * normdom1)))
      angle2 = np.degrees(np.arccos(np.matmul(np.transpose(dom2[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom2)))
      angle3 = np.degrees(np.arccos(np.matmul(np.transpose(dom3[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom3)))
      s1 = percent1 * (90 -angle1)
      s2 = percent2 * (90 -angle2)
      s3 = percent3 * (90 -angle3)
      if s1 > angle_min:
            angle_min = s1
            word = all_words[:,i]
      if s2 > angle_min:
            angle_min = s2
            word = all_words[:,i]
      if s3 > angle_min:
            angle_min = s3
            word = all_words[:,i]
  else:
    # repeated code if we have exactly 2 solutions remaining
    dom1 = np.matmul(U[:,0], Vh[0])*S[0]
    dom2 = np.matmul(U[:,1], Vh[1])*S[1]
    percent1 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom1))
    percent2 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom2))
    normdom1 = np.linalg.norm(dom1[:,0])
    normdom2 = np.linalg.norm(dom2[:,0])
    word = ""
    angle_vector = np.array([])
    angle_min = 0
    count = 0
    for i in range(0,all_words.shape[1]):
      angle1 = np.degrees(np.arccos(np.matmul(np.transpose(dom1[:,0]), (all_words[:,i]))/(np.linalg.norm(all_words[:,i]) * normdom1)))
      angle2 = np.degrees(np.arccos(np.matmul(np.transpose(dom2[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom2)))
      count += 1
      s1 = percent1 * (90 -angle1)
      s2 = percent2 * (90 -angle2)
      if s1 > angle_min:
            angle_min = s1
            word = all_words[:,i]
      if s2 > angle_min:
            angle_min = s2
            word = all_words[:,i]
  return vector_to_word(np.transpose(word))

In [ ]:
# We see in the output that this modified version of Bonthron's algorithm 
# still chooses SAINE as the starting guess.
all_words = np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
S = list(pd.read_csv("valid_solutions.csv")["word"])
solutionss = words_to_vectors(S)
guess = best_word(solutionss,all_words)
print(guess)

In [ ]:
# testing adapted Bonthron approach
S = list(pd.read_csv("valid_solutions.csv")["word"])
Sk = S.copy()
solved = []
attempts = []
Sk = rand.choices(Sk,k=1)
for secret in Sk:
    all_words = np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
    S = list(pd.read_csv("valid_solutions.csv")["word"])
    t = 1
    colors = []
    while t < 7 and colors != ["green"] * 5:
        if t == 1:
            guess = "saine"
        else:
            if len(S) == 1:
                guess = S[0]
            else:
                solutionss = words_to_vectors(S)
                guess = best_word(solutionss,all_words)
        print(guess)
        colors = coloring(secret,guess)
        if colors != ["green"] * 5:
            elim(S,guess,colors)
            t += 1
    solved.append(int(colors == ["green"] * 5))
    if solved[-1]:
        attempts.append(t)

print(sum(solved))
print(sum(solved)/len(solved))
print(sum(attempts)/len(attempts))

# 68 min 46 sec